In [10]:
import json
from census_response import getCensusResponse

In [31]:
#Estimate!!Total!!Population for whom poverty status is determined
pop_table = 'S1701_C01_001E'
#Estimate!!Below poverty level!!Population for whom poverty status is determined
poverty_table = 'S1701_C02_001E'
get_ls = [pop_table, poverty_table]

subject_table = 'https://api.census.gov/data/2018/acs/acs5/subject?'

#zip level request 
#Unable to filter request by state
geo = 'zip code tabulation area:*'

response = getCensusResponse(subject_table, get_ls, geo)


In [32]:
print(response.url)
response.text

https://api.census.gov/data/2018/acs/acs5/subject?get=NAME,S1701_C01_001E,S1701_C02_001E&for=zip%20code%20tabulation%20area:*&key=39a8090fc8ce00f64af354bf5286611974737d30


'[["NAME","S1701_C01_001E","S1701_C02_001E","zip code tabulation area"],\n["ZCTA5 43964","8626","1642","43964"],\n["ZCTA5 28216","50147","7238","28216"],\n["ZCTA5 28277","71558","2660","28277"],\n["ZCTA5 28278","27204","1257","28278"],\n["ZCTA5 28303","29057","5981","28303"],\n["ZCTA5 28314","57899","9818","28314"],\n["ZCTA5 28330","444","127","28330"],\n["ZCTA5 28347","1613","776","28347"],\n["ZCTA5 28351","4698","873","28351"],\n["ZCTA5 28357","2659","738","28357"],\n["ZCTA5 28369","2848","764","28369"],\n["ZCTA5 28371","7183","1331","28371"],\n["ZCTA5 28392","1502","624","28392"],\n["ZCTA5 28428","6492","603","28428"],\n["ZCTA5 28433","5362","1719","28433"],\n["ZCTA5 28435","2056","405","28435"],\n["ZCTA5 28450","2328","389","28450"],\n["ZCTA5 28461","19691","1471","28461"],\n["ZCTA5 28462","12659","1842","28462"],\n["ZCTA5 28537","138","40","28537"],\n["ZCTA5 28547","7250","832","28547"],\n["ZCTA5 28552","146","72","28552"],\n["ZCTA5 28557","15012","1663","28557"],\n["ZCTA5 28560",

In [33]:
#poverty_data is list of lists, where list elements are rows
#header row: name, population total, population poverty, fip state, fip county
poverty_data = response.json()
poverty_data[:5]

[['NAME', 'S1701_C01_001E', 'S1701_C02_001E', 'zip code tabulation area'],
 ['ZCTA5 43964', '8626', '1642', '43964'],
 ['ZCTA5 28216', '50147', '7238', '28216'],
 ['ZCTA5 28277', '71558', '2660', '28277'],
 ['ZCTA5 28278', '27204', '1257', '28278']]

In [43]:
#filter zipcodes
#IL zipcodes begin with numbers in zip_ls_IL
zip_ls_IL = ['60', '61', '62']

poverty_data_IL = []

for d in poverty_data[1:]:
    #print(d[3][:2])
    if d[3][:2] in zip_ls_IL:
        poverty_data_IL.append(d)
        
poverty_data_IL[:5]

[['ZCTA5 60546', '15641', '1094', '60546'],
 ['ZCTA5 60628', '66133', '17576', '60628'],
 ['ZCTA5 60480', '5370', '338', '60480'],
 ['ZCTA5 60554', '12127', '453', '60554'],
 ['ZCTA5 60641', '70691', '8891', '60641']]

In [49]:
#Json format
#{'countyfip': (countyfip = concatenation state and county fip numbers)
    #{'metric_one': 1234,
    #'metric_two': 5678}}
    
final_json = {}

for d in poverty_data_IL:
    #set variables to list elements
    name, pop_total_str, pop_poverty_str, zipcode = d
    #convert strings to ints
    pop_total_int = int(pop_total_str)
    pop_poverty_int = int(pop_poverty_str)
    #calculate percent poverty
    if pop_total_int != 0:
        pct_poverty = pop_poverty_int / pop_total_int * 100
    else:
        pct_poverty = 'Undefined'
    #create zip json
    zip_json = {'name_zip': name, 'population_total': pop_total_int, 'population_poverty': pop_poverty_int, 'percent_poverty': pct_poverty}
    #set county key to county json value
    final_json[zipcode] = zip_json

In [50]:
#save file
with open('final_jsons/acs5zippoverty_output.json', 'w') as f:
    json.dump(final_json, f)